In [17]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [24]:
df = pd.read_csv('../ml-data/cyberattack_detection/A2_1/training_data', sep=','
                       ,names=['Timestamp','Duration','Protocol','SrcIPAddress','SrcPort','Direction',
                               'DestIPAddress','DestPort','State','SrcTypeService','DestTypeService',
                               'TotalPackets','TotalBytes','SourceBytes'])

In [25]:
df.isna().any()

Timestamp          False
Duration           False
Protocol           False
SrcIPAddress       False
SrcPort             True
Direction          False
DestIPAddress      False
DestPort            True
State               True
SrcTypeService      True
DestTypeService     True
TotalPackets       False
TotalBytes         False
SourceBytes        False
dtype: bool

In [26]:
df.nunique()

Timestamp          13881753
Duration            3437518
Protocol                 14
SrcIPAddress          10811
SrcPort               97555
Direction                10
DestIPAddress        589070
DestPort              95865
State                   373
SrcTypeService            5
DestTypeService           4
TotalPackets          12327
TotalBytes           224601
SourceBytes           82174
dtype: int64

In [27]:
df_numerical = df[['Duration','TotalPackets','TotalBytes','TotalBytes','SourceBytes']]

In [28]:
df_numerical.head()

,Duration,TotalPackets,TotalBytes,TotalBytes,SourceBytes
0,67.205656,11,1216,1216,480
1,0.000000,3,59,59,58
2,0.000000,2,61,61,61
3,0.000000,2,64,64,58
4,0.000000,1,58,58,61


In [29]:
scaled_features = StandardScaler().fit_transform(df_numerical)
df_numerical = pd.DataFrame(scaled_features, index=df_numerical.index, columns=df_numerical.columns)

In [36]:
df_categorical = df[['Protocol','Direction','SrcTypeService','DestTypeService']]

In [11]:
training_data[training_data['SrcPort'].isnull()]

,Timestamp,Duration,Protocol,SrcIPAddress,SrcPort,Direction,DestIPAddress,DestPort,State,SrcTypeService,DestTypeService,TotalPackets,TotalBytes,SourceBytes
3064,2012/12/01 13:11:26.252055,3488.878144,pim,162.217.94.6,NaN,->,150.184.7.6,NaN,INT,0.0,NaN,191,14239,13359
3507,2012/12/01 13:11:29.310824,0.000396,icmp,152.205.93.89,NaN,<-,146.204.94.31,0xcb07,RED,NaN,0.0,3,194,0
3639,2012/12/01 13:11:30.311807,0.000390,icmp,152.205.93.89,NaN,<-,146.204.94.31,0xcb07,ECR,NaN,0.0,2,196,0
3749,2012/12/01 13:11:31.315594,0.000010,icmp,152.205.93.89,NaN,<-,146.204.94.31,0xcb07,ECR,NaN,0.0,3,193,0
3859,2012/12/01 13:11:31.736798,3473.586043,arp,169.177.94.33,NaN,who,229.173.94.6,NaN,CON,NaN,NaN,110,6921,3316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13878759,2012/12/10 15:09:47.345212,0.000000,igmp,162.217.94.6,NaN,->,155.89.7.6,NaN,INT,192.0,NaN,1,60,61
13878767,2012/12/10 15:09:47.672333,0.000000,igmp,157.196.94.31,NaN,->,202.190.9.243,NaN,INT,0.0,NaN,1,62,60
13878801,2012/12/10 15:09:48.279993,0.000000,igmp,174.114.94.3,NaN,->,145.162.9.246,NaN,INT,0.0,NaN,2,60,57
13878802,2012/12/10 15:09:48.280647,0.000000,igmp,174.114.94.3,NaN,->,192.67.8.15,NaN,INT,0.0,NaN,1,57,62


In [10]:
nan_rows

,Timestamp,Duration,Protocol,SrcIPAddress,SrcPort,Direction,DestIPAddress,DestPort,State,SrcTypeService,DestTypeService,TotalPackets,TotalBytes,SourceBytes
3064,2012/12/01 13:11:26.252055,3488.878144,pim,162.217.94.6,NaN,->,150.184.7.6,NaN,INT,0.0,NaN,191,14239,13359
3507,2012/12/01 13:11:29.310824,0.000396,icmp,152.205.93.89,NaN,<-,146.204.94.31,0xcb07,RED,NaN,0.0,3,194,0
3639,2012/12/01 13:11:30.311807,0.000390,icmp,152.205.93.89,NaN,<-,146.204.94.31,0xcb07,ECR,NaN,0.0,2,196,0
3749,2012/12/01 13:11:31.315594,0.000010,icmp,152.205.93.89,NaN,<-,146.204.94.31,0xcb07,ECR,NaN,0.0,3,193,0
3859,2012/12/01 13:11:31.736798,3473.586043,arp,169.177.94.33,NaN,who,229.173.94.6,NaN,CON,NaN,NaN,110,6921,3316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13878759,2012/12/10 15:09:47.345212,0.000000,igmp,162.217.94.6,NaN,->,155.89.7.6,NaN,INT,192.0,NaN,1,60,61
13878767,2012/12/10 15:09:47.672333,0.000000,igmp,157.196.94.31,NaN,->,202.190.9.243,NaN,INT,0.0,NaN,1,62,60
13878801,2012/12/10 15:09:48.279993,0.000000,igmp,174.114.94.3,NaN,->,145.162.9.246,NaN,INT,0.0,NaN,2,60,57
13878802,2012/12/10 15:09:48.280647,0.000000,igmp,174.114.94.3,NaN,->,192.67.8.15,NaN,INT,0.0,NaN,1,57,62


In [23]:
training_data = training

In [24]:
training_data.head()

,Timestamp,Duration,Protocol,SrcIPAddress,SrcPort,Direction,DestIPAddress,DestPort,State,SrcTypeService,DestTypeService,TotalPackets,TotalBytes,SourceBytes
0,2012/12/01 03:25:11.124065,67.205656,tcp,181.74.92.157,4832,->,203.187.236.103,3389,FSPA_FSPA,0.0,0.0,11,1216,480
1,2012/12/01 03:25:23.962955,0.000000,tcp,181.74.92.157,4842,->,128.129.8.3,135,S_,0.0,NaN,3,59,58
2,2012/12/01 03:25:23.989079,0.000000,tcp,181.74.92.157,4836,->,133.185.92.5,135,S_,0.0,NaN,2,61,61
3,2012/12/01 03:25:23.994181,0.000000,tcp,181.74.92.157,4839,->,199.184.92.4,135,S_,0.0,NaN,2,64,58
4,2012/12/01 03:25:24.028562,0.000000,tcp,181.74.92.157,4833,->,133.83.93.101,135,S_,0.0,NaN,1,58,61


In [27]:
training_data.to_csv('/Applications/Splunk/etc/apps/Splunk_ML_Toolkit/lookups/A1_training_data.csv', sep=',', index=False)